In [0]:
#Install Java Runtime 8
import os       #importing os to set environment variable
!apt-get remove -y java*
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

In [0]:
#Initialize Spark Runtime Environment
!export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/jre
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import SQLContext

spark = SparkSession.builder \
   .master("local") \
   .appName("Wordcount & Friends Tutorial") \
   .config("spark.executor.memory", "2gb") \
   .getOrCreate()
   
sc = spark.sparkContext
sqlContext = SQLContext(sc)

sc.version

PART-1:  "WORDCOUNT" EXAMPLE USING MAPREDUCE

In [0]:
# Fetch the text file for wordcount example
!wget https://raw.githubusercontent.com/ibarabasi/wordcount/master/wordcount
!cat wordcount

In [0]:
#Simple example to read text file
rdd0 = sc.textFile("wordcount")
rdd0.take(20)

In [0]:
word_counts = rdd0.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: (a + b)).map(lambda x:(x[1],x[0]))

word_counts.take(30)


PART-2:  "COMMON FRIENDS" EXAMPLE USINg MAPREDUCE

In [0]:
# Load data from github
!wget "https://raw.githubusercontent.com/ibarabasi/wordcount/master/friends"
rdd = sc.textFile("friends")
!cat friends

In [0]:
# Print the RDD content
rdd.take(20)

In [0]:
rdd1=rdd.map(lambda x: x.split()).union(rdd.map(lambda x: x.split()[::-1]))
# Bring my friend list to local
lst = rdd1.filter(lambda x: x[0] == 'me').map(lambda x: x[1]).collect()
# Build the second pair RDD
rdd2 = rdd1.filter(lambda x: x[0] in lst).map(lambda x: x[1]). \
    filter(lambda x: x != 'me' and x not in lst). \
    map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b). \
    map(lambda x: (x[1], x[0])).sortByKey(ascending = False)
# Bring the result to local since the sample is small
for x, y in rdd2.collect():
   print ("The stranger {} has {} common friends with me".format(y, x))
